In [ ]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [ ]:
# Download training data from open datasets
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

100%|██████████| 26421880/26421880 [00:01<00:00, 13832327.27it/s]


Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 29515/29515 [00:00<00:00, 303832.13it/s]


Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 4422102/4422102 [00:00<00:00, 5496114.92it/s]


Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 5148/5148 [00:00<00:00, 3727304.85it/s]


Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw



In [ ]:
# Download training data from open datasets
mnist_training_data = datasets.MNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

mnist_test_data = datasets.MNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 9912422/9912422 [00:00<00:00, 49507565.92it/s]


Extracting data/MNIST/raw/train-images-idx3-ubyte.gz to data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 28881/28881 [00:00<00:00, 1765594.80it/s]

Extracting data/MNIST/raw/train-labels-idx1-ubyte.gz to data/MNIST/raw



Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 1648877/1648877 [00:00<00:00, 12227574.00it/s]


Extracting data/MNIST/raw/t10k-images-idx3-ubyte.gz to data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 4542/4542 [00:00<00:00, 2462581.28it/s]

Extracting data/MNIST/raw/t10k-labels-idx1-ubyte.gz to data/MNIST/raw



In [ ]:
batch_size = 64

# Create data loaders
mnist_train_dataloader = DataLoader(mnist_training_data, batch_size=batch_size)
mnist_test_dataloader = DataLoader(mnist_test_data, batch_size=batch_size)

for  X, y in mnist_test_dataloader:
  print(f"Shape of X [N, C, H, W]: {X.shape}")
  print(f"Shape of y: {y.shape}, {y.dtype}")
  break

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]), torch.int64


In [ ]:
batch_size = 64

# Create data loaders
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for  X, y in test_dataloader:
  print(f"Shape of X [N, C, H, W]: {X.shape}")
  print(f"Shape of y: {y.shape}, {y.dtype}")
  break

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]), torch.int64


In [ ]:
# Creating Models

# To define a neural network in PyTorch,
# we create a class that inherits from nn.Module.
# We define the layers of the network in the __init__ function
# and specify how data will pass through the network in the forward function.
# To accelerate operations in the neural network,
# we move it to the GPU or MPS if available.

In [ ]:
# Get cpu, gpu or mps device for training
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using device {device}")

Using device cpu


In [ ]:
# Define model
class NeuralNetwork(nn.Module):
  def __init__(self):
    super().__init__()
    self.flatten = nn.Flatten()
    self.softmax_relu_stack = nn.Sequential(
        nn.Linear(28*28, 512),
        nn.ReLU(),
        nn.Linear(512, 512),
        nn.ReLU(),
        nn.Linear(512, 10),
        # nn.Softmax()
    )

  def forward(self, x):
    x = self.flatten(x)
    logits = self.softmax_relu_stack(x)
    return logits

model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (softmax_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [ ]:
# Optimizing the model parameters
# To train a model we need
# 1. a loss function
# 2. an optimizer

loss_fn = nn.CrossEntropyLoss()
# optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
print(f"Loss function = {loss_fn}")
print(f"Optimizer = {optimizer}")

Loss function = CrossEntropyLoss()
Optimizer = Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: None
    lr: 0.001
    maximize: False
    weight_decay: 0
)


In [ ]:
# In a single training loop,
# the model makes predictions on the training dataset (fed to it in batches)
# and brackpropagates the prediction error to adjust the model's parameters

In [ ]:
def train(dataloader, model, loss_fn, optimizer):
  size = len(dataloader.dataset)
  print(f"Dataloader type = {type(dataloader)}")
  model.train()
  for batch, (X, y) in enumerate(dataloader):
    X, y = X.to(device), y.to(device)

    # Compute prediction error
    pred = model(X)
    loss = loss_fn(pred, y)

    # Backpropagation
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()

    if batch%100 == 0:
      loss, current = loss.item(), (batch+1)*len(X)
      print(f"loss: {loss:>7f} [{current:>5d}/ {size:>5d}]")

In [ ]:
# We also check the model's performance against the tes dataset to ensure it is learning

def test(dataloader, model, loss_fn):
  size = len(dataloader.dataset)
  num_batches = len(dataloader)
  model.eval()
  test_loss, correct = 0, 0
  with torch.no_grad():
    for X, y in dataloader:
      X, y = X.to(device), y.to(device)
      pred = model(X)
      test_loss += loss_fn(pred, y).item()
      correct += (pred.argmax(1) == y).type(torch.float).sum().item()
  test_loss /= num_batches
  correct /= size
  print(f"Test Error: \n Accuracy: {(100*correct) :>0.1f}%, Avg loss: {test_loss:>8f}\n")

In [ ]:
# The training process is conducted over several epochs/ iterations.
# During each epoch, the model learns parameters to make better predictions.
# We print the model's accuracy and loss at each epoch; we'd like to see
# the accuracty increase and loss decreasing with every epoch

epochs = 5
for t in range(epochs):
  print(f"Epoch {t+1}\n-------------------")
  train(mnist_train_dataloader, model, loss_fn, optimizer)
  test(mnist_test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------
Dataloader type = <class 'torch.utils.data.dataloader.DataLoader'>
loss: 2.312617 [   64/ 60000]
loss: 0.281910 [ 6464/ 60000]
loss: 0.202289 [12864/ 60000]
loss: 0.241962 [19264/ 60000]
loss: 0.133777 [25664/ 60000]
loss: 0.347707 [32064/ 60000]
loss: 0.124561 [38464/ 60000]
loss: 0.274744 [44864/ 60000]
loss: 0.337179 [51264/ 60000]
loss: 0.138242 [57664/ 60000]
Test Error: 
 Accuracy: 95.6%, Avg loss: 0.138237

Epoch 2
-------------------
Dataloader type = <class 'torch.utils.data.dataloader.DataLoader'>
loss: 0.087840 [   64/ 60000]
loss: 0.115749 [ 6464/ 60000]
loss: 0.092728 [12864/ 60000]
loss: 0.110904 [19264/ 60000]
loss: 0.048080 [25664/ 60000]
loss: 0.137385 [32064/ 60000]
loss: 0.039759 [38464/ 60000]
loss: 0.153590 [44864/ 60000]
loss: 0.131135 [51264/ 60000]
loss: 0.083268 [57664/ 60000]
Test Error: 
 Accuracy: 95.8%, Avg loss: 0.136621

Epoch 3
-------------------
Dataloader type = <class 'torch.utils.data.dataloader.DataLoader'>
loss: 0.08493

In [ ]:
# Save the model
#  serialize the internal state dictionary - containing the model parameters

torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model Steat to model.pth")

Saved PyTorch Model Steat to model.pth


In [ ]:
# Loading models
# Re-create model structure and loading the state dictionary into it

model = NeuralNetwork().to(device)
model.load_state_dict(torch.load("model.pth", weights_only=True))

<All keys matched successfully>

In [ ]:
# Make new predictions

# classes = [

# ]
x, y = mnist_test_data[0][0], mnist_test_data[0][1]
print(f"X, y = {X[0][0][0]}, {y}")

X, y = tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.]), 7


In [ ]:
x, y = test_data[0][0], test_data[0][1]
print(f"X, y = {X[0][0][0]}, {y}")

X, y = tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.]), 9


In [ ]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_data[0][0], test_data[2][1]
with torch.no_grad():
    x = x.to(device)
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')



Predicted: "Pullover", Actual: "Trouser"


In [ ]:
classes = list(range(0,10))
# print(f"Classes = {classes}")
right_ones, wrong_ones = 0, 0
softmax_fn = torch.nn.Softmax()
model.eval()
for i in range(10,20):
  x, y = mnist_test_data[i][0], mnist_test_data[i][1]
  with torch.no_grad():
      x = x.to(device)
      pred = model(x)
      pred = softmax_fn(pred)
      # print(f"Pred = {pred}")
      predicted, actual = classes[pred[0].argmax(0)], classes[y]
      # print(f'Predicted: "{predicted}", Actual: "{actual}"')
      if predicted==actual:
        right_ones += 1
print(f"Right % = {(right_ones/10)*100}")

Right % = 100.0


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py:1553: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)


In [ ]:
print(f"Length of training data = {len(training_data)}")
print(f"Length of test data = {len(test_data)}")

print(f"Length of mnist training data = {len(mnist_training_data)}")
print(f"Length of mnist test data = {len(mnist_test_data)}")

Length of training data = 60000
Length of test data = 10000
Length of mnist training data = 60000
Length of mnist test data = 10000
